## Prepare data reader and model execution context

In [1]:

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.blockbatch.construct import construct_data_readers
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "refined_traces"

# Define the input data range.
readers: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=[model_name],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS,
)


# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, aux_views = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0] # use the first reader
)


2024-12-28 11:49:19 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25 process=83116
2024-12-28 11:49:19 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=83116
2024-12-28 11:49:19 [debug    ] loaded vault: 18 items         filename=vault.py lineno=76 process=83116
2024-12-28 11:49:20 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37 process=83116
2024-12-28 11:49:20 [info     ] prepared 22 input batches.     filename=byblock.py lineno=85 process=83116
2024-12-28 11:49:20 [info     ] initialized duckdb at /var/folders/tz/g2vwn0qx5ll32j7jfsshhps40000gn/T/t8aylo_p/op-analytics.duck.db filename=client.py lineno=61 process=83116
2024-12-28 11:49:20 [info     ] reading dataset='ingestion/blocks_v1' using 1/1 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/blocks_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=83116
2024-12-28 11:49:20 [info     ] r

## Execute the model

It's up to you how the model manipulates the data. 

Develop the various steps in notebook cells and then copy the final code over to the model function.

In [2]:
refined_txs_table = aux_views["refined_transactions_fees"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_blocks": input_datasets["ingestion/blocks_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
    }
)

refined_traces_projection = aux_views["refined_traces/traces_projection"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": input_datasets["ingestion/traces_v1"].as_subquery(),
    },
)

traces_amortized = aux_views["refined_traces/traces_amortized"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "refined_traces_projection": refined_traces_projection,
    },
)

traces_txs_join = aux_views["refined_traces/traces_txs_join"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "traces_amortized": traces_amortized,
        "refined_transactions_fees": refined_txs_table,
    },
)

2024-12-28 11:49:23 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=83116 template=refined_transactions_fees
2024-12-28 11:49:31 [info     ] duck db size: 8.4MB            filename=client.py lineno=33 process=83116
2024-12-28 11:49:31 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=83116 template=refined_traces/traces_projection
2024-12-28 11:49:40 [info     ] duck db size: 59.0MB           filename=client.py lineno=33 process=83116
2024-12-28 11:49:40 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=83116 template=refined_traces/traces_amortized
2024-12-28 11:49:41 [info     ] duck db size: 97.5MB           filename=client.py lineno=33 process=83116
2024-12-28 11:49:41 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=83116 template=refined_traces/traces_txs_join
2024-12-28 11:49:41 [info     ] duck db size: 175.1MB          filename=clie

In [3]:
ctx.client.sql("""
SELECT * FROM refined_traces__traces_txs_join
WHERE transaction_hash = '0x455d3fa505b6ff90bdb6385735c258a399d21c9d46c0a457c9a10505e09e925b'
ORDER BY trace_address
""").show(max_rows=100)

┌────────────┬─────────┬──────────┬─────────┬──────────────┬─────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────────┬────────────────┬───────────────┬────────────┬──────────────┬─────────┬─────────────────┬───────────────┬─────────────┬──────────────────────┬───────────────────┬────────────────────────┬──────────────────────────┬──────────────────────────────────┬───────────────────────────────────────┬───────────────────────────────────────────┬──────────────────────────────────────────────┬──────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬─────────────────────────────────────┬─────────────────────────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────────────────────────

In [2]:
refined_txs = aux_views["refined_transactions_fees"].render(
    context={
        "raw_blocks": input_datasets["ingestion/blocks_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
    }
)

refined_txs_table = aux_views["refined_transactions_fees"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_blocks": input_datasets["ingestion/blocks_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
    }
)

refined_txs_projection = aux_views["refined_traces/txs_projection"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "refined_transactions_fees": f"({refined_txs})",
    },
)


refined_traces_projection = aux_views["refined_traces/traces_projection"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": input_datasets["ingestion/traces_v1"].as_subquery(),
    },
)

traces_amortized = aux_views["refined_traces/traces_amortized"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "refined_traces_projection": refined_traces_projection,
    },
)


2024-12-28 06:31:20 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=50338 template=refined_transactions_fees
2024-12-28 06:31:20 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=50338 template=refined_transactions_fees
2024-12-28 06:31:27 [info     ] duck db size: 8.4MB            filename=client.py lineno=33 process=50338
2024-12-28 06:31:27 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=50338 template=refined_traces/txs_projection
2024-12-28 06:31:33 [info     ] duck db size: 8.4MB            filename=client.py lineno=33 process=50338
2024-12-28 06:31:33 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=50338 template=refined_traces/traces_projection
2024-12-28 06:31:42 [info     ] duck db size: 63.2MB           filename=client.py lineno=33 process=50338
2024-12-28 06:31:42 [info     ] Rendering query                filename=querybuilde

In [4]:
aux_views["refined_traces/traces_txs_join"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "traces_amortized": traces_amortized,
        "refined_transactions_fees": refined_txs_table,
    },
)

2024-12-28 06:35:17 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=50338 template=refined_traces/traces_txs_join
2024-12-28 06:35:17 [info     ] duck db size: 178.5MB          filename=client.py lineno=33 process=50338


'refined_traces__traces_txs_join'

In [5]:
ctx.client.sql("""
SELECT * FROM refined_traces__traces_txs_join
WHERE transaction_hash = '0x455d3fa505b6ff90bdb6385735c258a399d21c9d46c0a457c9a10505e09e925b'
ORDER BY trace_address
""").show(max_rows=100)

┌────────────┬─────────┬──────────┬─────────┬──────────────┬─────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────────┬────────────────┬───────────────┬────────────┬──────────────┬─────────┬─────────────────┬───────────────┬─────────────┬──────────────────────┬───────────────────┬────────────────────────┬──────────────────────────┬──────────────────────────────────┬───────────────────────────────────────┬───────────────────────────────────────────┬──────────────────────────────────────────────┬──────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬─────────────────────────────────────┬─────────────────────────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────────────────────────

In [16]:
ctx.client.sql(f"""DESCRIBE {refined_txs_table}""").show(max_rows=100)

┌─────────────────────────────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│             column_name             │  column_type   │  null   │   key   │ default │  extra  │
│               varchar               │    varchar     │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                                  │ DATE           │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                               │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id                            │ INTEGER        │ YES     │ NULL    │ NULL    │ NULL    │
│ network                             │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ nonce                               │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_index                   │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ from_address                

In [15]:
ctx.client.sql(f"""DESCRIBE {refined_txs_projection}""").show(max_rows=100)

┌───────────────────────────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│            column_name            │  column_type   │  null   │   key   │ default │  extra  │
│              varchar              │    varchar     │ varchar │ varchar │ varchar │ varchar │
├───────────────────────────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ tx_l2_gas_used                    │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ tx_l1_gas_used_unified            │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ tx_estimated_size                 │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ tx_from_address                   │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ tx_to_address                     │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ tx_fee_native                     │ DECIMAL(38,19) │ YES     │ NULL    │ NULL    │ NULL    │
│ tx_l1_fee_native                  │ DECIMAL(38,1

In [5]:
print(";\n\n".join([stmt1, stmt2]))

CREATE OR REPLACE TABLE refined_traces__txs_projection AS
SELECT
  -- Raw Transaction Fields
  t.l2_gas_used AS tx_l2_gas_used
  , t.l1_gas_used_unified AS tx_l1_gas_used_unified
  , t.estimated_size AS tx_estimated_size
  , t.from_address AS tx_from_address
  , t.to_address AS tx_to_address
  -- Computed Transaction Fee Fields
  , t.tx_fee_native
  , t.l1_fee_native AS tx_l1_fee_native
  , t.l2_fee_native AS tx_l2_fee_native
  , t.l2_priority_fee_native AS tx_l2_priority_fee_native
  , t.l2_base_fee_native AS tx_l2_base_fee_native
  , t.l2_legacy_extra_fee_native AS tx_l2_legacy_extra_fee_native
  -- Computed Transaction Gas Price Fields
  , t.l2_gas_price_gwei AS tx_l2_gas_price_gwei
  , t.l2_base_gas_price_gwei AS tx_l2_base_gas_price_gwei
  , t.l2_priority_gas_price_gwei AS tx_l2_priority_gas_price_gwei
  , t.l2_legacy_extra_gas_price_gwei AS tx_l2_legacy_extra_gas_price_gwei
  , t.l1_base_gas_price_gwei AS tx_l1_base_gas_price_gwei
  , t.l1_blob_base_gas_price_gwei AS tx_l1_blob_b

In [11]:
from op_analytics.coreutils.threads import run_concurrently

run_concurrently(ctx.client.sql, [stmt1, stmt2])
ctx.client.sql("SHOW TABLES")
ctx.report_size()

2024-12-20 12:34:40 [info     ] duck db size: 113.5MB          filename=client.py lineno=33 process=91188


In [8]:
ctx.client.sql(stmt1)
ctx.client.sql(stmt2)
ctx.client.sql("SHOW TABLES")
ctx.report_size()

2024-12-20 12:33:04 [info     ] duck db size: 113.5MB          filename=client.py lineno=33 process=91188


In [9]:
ctx.client.sql(";\n\n".join([stmt1, stmt2]))
ctx.client.sql("SHOW TABLES")
ctx.report_size()

2024-12-20 12:33:14 [info     ] duck db size: 114.0MB          filename=client.py lineno=33 process=91188


In [7]:
ctx.report_size()

2024-12-20 12:31:46 [info     ] duck db size: 64.8MB           filename=client.py lineno=33 process=91188


In [ ]:


refined_txs_statement = aux_views["refined_transactions_fees"].create_table_statement(
    duckdb_context=ctx,
    template_parameters={
        "raw_blocks": f"(\n{blocks}\n)" ,
        "raw_transactions": f"(\n{transactions}\n)" ,
    },
)



refined_traces = AuxiliaryView("refined_traces_fees").create_table_statement(
        duckdb_context=ctx,
        template_parameters={
            "raw_traces": traces,
            "refined_transactions_fees": refined_txs,
        },
    )

In [ ]:
# Create a table with the refined transactions

blocks_view = input_datasets["ingestion/blocks_v1"].create_view()
transactions_view = input_datasets["ingestion/transactions_v1"].create_view()

refined_txs = aux_views["refined_transactions_fees"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_blocks": blocks_view,
        "raw_transactions": transactions_view,
    },
)

In [ ]:
from op_analytics.datapipeline.models.compute.auxview import AuxiliaryView

traces_table = input_datasets["ingestion/traces_v1"].create_table(
    additional_sql="ORDER BY block_number, transaction_hash, trace_address"
)

refined_traces = AuxiliaryView("refined_traces_fees").create_table(
        duckdb_context=ctx,
        template_parameters={
            "raw_traces": traces_table,
            "refined_transactions_fees": refined_txs,
        },
    )



In [ ]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_client=duckdb_client,
    model=model_name,
    data_reader=batch,
    limit_input_parquet_files=1
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

## Verify model results - Traces

In [ ]:
duckdb_client.sql("SELECT * FROM daily_traces_tr_to_v1 ORDER BY count_transactions_called DESC LIMIT 10")

### Test Transaction Metrics

In [ ]:
duckdb_client.sql("""
                  SELECT trace_to_address,
                    count_transactions_called_with_internal_type_call, count_transactions_called_with_internal_type_call_or_delegate,
                    count_transactions_called
                  FROM daily_traces_tr_to_v1
                  ORDER BY count_transactions_called_with_internal_type_call DESC
                  LIMIT 10
                  """)

### Test Gas Used Metrics

In [ ]:
duckdb_client.sql("""
                  SELECT
                    trace_to_address,
                  
                    sum_trace_gas_used_minus_subtraces_tx_success_called_with_internal_type_call,
                    sum_tx_l2_gas_used_amortized_by_call_tx_success_called_with_internal_type_call,
                  
                    sum_tx_l2_fee_native_minus_subtraces_tx_success_called_with_internal_type_call
                    sum_tx_l2_fee_native_amortized_by_call_tx_success_called_with_internal_type_call,
                    sum_tx_fee_native_amortized_by_call_tx_success_called_with_internal_type_call,
                  
                    count_transactions_called_with_internal_type_call,
                    count_transactions_called_with_internal_type_call_or_delegate,
                    count_transactions_called
                  
                  FROM daily_traces_tr_to_v1
                  ORDER BY sum_trace_gas_used_minus_subtraces_tx_success_called_with_internal_type_call DESC
                  LIMIT 10
                  """)

In [ ]:
duckdb_client.sql("""
    SELECT * FROM refined_traces_fees_v1
                  where transaction_hash = '0xc620133c2339f36d8bfae889ea29e9986a70182f7bbe3380d0622f3801619eda'
                  AND block_number = 128145924
                  ORDER BY trace_address ASC
                  LIMIT 10
                  """)

### Check the data output size

In [ ]:
duckdb_client.sql("SELECT COUNT(*) AS interm_num_calls, COUNT(DISTINCT transaction_hash) AS num_txs, COUNT(DISTINCT trace_to_address) AS num_trace_tos FROM refined_traces_fees_v1")

In [ ]:
duckdb_client.sql("SELECT COUNT(*) AS num_rows, COUNT(DISTINCT transaction_hash) AS num_txs, COUNT(DISTINCT trace_to_address) AS num_trace_tos FROM aggregated_traces_tr_to_hash_v1")

In [ ]:
duckdb_client.sql("SELECT COUNT(*) AS num_rows, COUNT(DISTINCT trace_to_address) AS num_trace_tos FROM daily_traces_tr_to_v1")

### Get table schema

In [ ]:
duckdb_client.sql("DESCRIBE daily_traces_tr_to_v1")
